In [21]:
import pandas as pd
import requests as platypus
from bs4 import BeautifulSoup

## First, we'll read in the data output by the scraper and do any cleaning that didn't survive the "to_csv" part of the last notebook.  

In [2]:
df = pd.read_csv('AllHoldings.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390168 entries, 0 to 390167
Data columns (total 12 columns):
manager            390168 non-null object
filing_date        390168 non-null object
report_date        390168 non-null object
sh_name            390168 non-null object
sh_class           390168 non-null object
cusip              390168 non-null object
val1000            390168 non-null int64
share_count        390168 non-null int64
share_or_prin      390168 non-null object
discretion         390168 non-null object
sole_vote_amt      390168 non-null int64
shared_vote_amt    390168 non-null int64
dtypes: int64(4), object(8)
memory usage: 35.7+ MB


In [4]:
df['report_date'] = pd.to_datetime(df.report_date)

In [5]:
df['filing_date'] = pd.to_datetime(df.filing_date)

## The next step is to return only the top 20 holdings, aggregated by date and Cusip/Share.   

In [7]:
summed_df = df.groupby(['report_date', 'cusip', 'sh_name'])['report_date', 'cusip', 'sh_name', 'val1000'].sum().reset_index()
# grouped_df.sum().nlargest(multiplier,'val1000').sort_values(by = ['report_date','val1000'], ascending=False)

# df.groupby('State')['Population'].apply(lambda grp: grp.nlargest(2).sum())

In [8]:
summed_df.set_index('report_date')


,cusip,sh_name,val1000
report_date,,,
2013-06-30,000360206,AAON INC,1179
2013-06-30,000361105,AAR CORP,2039
2013-06-30,000375204,ABB LTD,4802
2013-06-30,00081T108,ACCO BRANDS CORP,65
2013-06-30,000957100,ABM INDS INC,1731
2013-06-30,00101J106,THE ADT CORPORATION,12867
2013-06-30,001031103,AEP INDS INC,536
2013-06-30,00104Q107,AFC ENTERPRISES INC,4421
2013-06-30,001055102,AFLAC INC,75176


In [9]:
looper = summed_df.report_date.value_counts().index
top_df = pd.DataFrame(columns=['report_date', 'cusip', 'sh_name', 'val1000'])

In [14]:
for x in looper:
    top_df = top_df.append(summed_df[(summed_df.report_date == x)].nlargest(20, 'val1000'))

In [17]:
top_df.sort_values(['report_date', 'val1000'], ascending=[False, False])

,report_date,cusip,sh_name,val1000
135379,2018-06-30,037833100,APPLE INC,54835353
139280,2018-06-30,78462F103,SPDR S&P 500 ETF TR,29281034
140339,2018-06-30,949746101,WELLS FARGO & CO NEW,25059643
135568,2018-06-30,060505104,BANK AMER CORP,21612437
138013,2018-06-30,500754106,KRAFT HEINZ CO,20906754
136087,2018-06-30,191216100,COCA COLA CO,18764843
135270,2018-06-30,023135106,AMAZON COM INC,18040550
135292,2018-06-30,025816109,AMERICAN EXPRESS CO,15349606
137423,2018-06-30,451100101,ICAHN ENTERPRISES LP,11826854
138332,2018-06-30,594918104,MICROSOFT CORP,10012998


In [16]:
top_df.to_csv('cusip_date.csv')

In [38]:
stocks = top_df.sh_name.unique()
url='https://www.google.com/search?q='

In [39]:
for stock in stocks:
    x= stock.replace('&', 'and')
    x= x.replace(' ', '+')
    stock2.append(x)

In [41]:
stock2[:5]

['WELLS+FARGO+and+CO+NEW',
 'KRAFT+HEINZ+CO',
 'COCA+COLA+CO',
 'INTERNATIONAL+BUSINESS+MACHS',
 'AMERICAN+EXPRESS+CO']

In [19]:
for stock in stock2[:4]:
    duckbill = platypus.get(url+stock).text
    soup = BeautifulSoup(duckbill, 'lmxl')
    

63


In [42]:
stock_list = []
for stock in stocks:
    stock_list.append(stock)

In [46]:
stocks

array(['WELLS FARGO & CO NEW', 'KRAFT HEINZ CO', 'COCA COLA CO',
       'INTERNATIONAL BUSINESS MACHS', 'AMERICAN EXPRESS CO', 'APPLE INC',
       'ICAHN ENTERPRISES LP', 'SPDR S&P 500 ETF TRUST', 'MICROSOFT CORP',
       'PROCTER & GAMBLE CO', 'PHILLIPS 66', 'US BANCORP DEL',
       'AIR PRODS & CHEMS INC', 'VALEANT PHARMACEUTICALS INTL',
       'AT&T INC', 'AMGEN INC', 'WAL MART STORES INC', 'PEPSICO INC',
       'PFIZER INC', 'DIRECTV', 'VANGUARD INTL EQUITY INDEX F',
       'EBAY INC', 'GENERAL MTRS CO', 'AMERICAN INTL GROUP INC',
       'ALLERGAN PLC', 'AMAZON COM INC', 'MEDTRONIC PLC', 'ALLERGAN INC',
       'MERCK & CO INC NEW', 'SPDR S&P 500 ETF TR',
       'CHARTER COMMUNICATIONS INC N', 'E M C CORP MASS',
       'JPMORGAN CHASE & CO', 'DAVITA HEALTHCARE PARTNERS I',
       'CHEVRON CORP NEW', 'BAXTER INTL INC', 'BANK AMER CORP',
       'INVESCO QQQ TR', 'FACEBOOK INC', 'ALPHABET INC',
       'ALIBABA GROUP HLDG LTD', 'WELLS FARGO CO NEW', 'NETFLIX INC',
       'BOOKING HLDGS 